classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from PIL.Image import LANCZOS
from config import *

import copy
import math


In [2]:
torch.cuda.is_available()

True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
from data.taskers import gen_tasks
from src.zoo.archs import CVAE
from src.zoo.lpo_utils import proto_distr, classify, set_sets, accuracy, inner_adapt_lpo

#from src.utils import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../mini_imagenet', mode='train', n_ways=5, k_shots=3, q_shots=10, image_transforms=None)

In [6]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False, remove_linear=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        if remove_linear:
            self.fc = None
        else:
            self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x, feature=False):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        if self.fc is None:
            if feature:
                return x, None
            else:
                return x
        if feature:
            x1 = self.fc(x)
            return x, x1
        x = self.fc(x)
        return x


In [7]:
resnet18 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=64, remove_linear=True)

In [8]:
checkpoint = torch.load('/home/anuj/Desktop/Work/TU_Delft/research/implement/results/mini/softmax/resnet18/model_best.pth.tar')

RuntimeError: CUDA error: out of memory

In [21]:
model_dict = resnet18.state_dict()
params = checkpoint['state_dict']
params = {k: v for k, v in params.items() if k in model_dict}
model_dict.update(params)
resnet18.load_state_dict(model_dict)

<All keys matched successfully>

In [26]:
ttask = train_tasks.sample()
data, label = ttask
data.shape

torch.Size([65, 3, 84, 84])

In [29]:
resnet18(data)

tensor([[1.1363, 1.3802, 0.9376,  ..., 1.0885, 0.8399, 0.6831],
        [0.7737, 0.6379, 0.8058,  ..., 0.9359, 0.9687, 0.8883],
        [0.5301, 0.4768, 0.4925,  ..., 0.7668, 0.5795, 0.9291],
        ...,
        [0.5636, 0.8997, 0.7224,  ..., 0.8275, 0.6633, 0.9512],
        [0.4752, 0.2371, 0.4084,  ..., 0.6061, 0.7444, 0.7838],
        [1.0797, 0.8569, 0.6350,  ..., 0.8993, 0.7728, 0.8118]])

In [28]:
for p in resnet18.parameters():
    p.requires_grad = False

In [24]:
resnet34 = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=64, remove_linear=True)
checkpoint = torch.load('/home/anuj/Desktop/Work/TU_Delft/research/implement/results/mini/softmax/resnet34/checkpoint.pth.tar')
model_dict = resnet18.state_dict()
params = checkpoint['state_dict']
params = {k: v for k, v in params.items() if k in model_dict}
model_dict.update(params)
resnet34.load_state_dict(model_dict)

FileNotFoundError: [Errno 2] No such file or directory: '/home/anuj/Desktop/Work/TU_Delft/research/implement/results/mini/softmax/resnet34/checkpoint.pth.tar'

In [ ]:
resnet50 = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=64, remove_linear=True)

In [ ]:
resnet101 = ResNet(Bottleneck, [3, 4, 23, 3], num_classes=64, remove_linear=True)

In [ ]:
resnet152 = ResNet(Bottleneck, [3, 8, 36, 3], num_classes=64, remove_linear=True)